In [1]:
import os
import requests
import numpy as np
import duckdb
import polars as pl
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [3]:
from typing import Dict, Optional, List, Any

In [4]:
from anthropic.types import Message, TextBlock, ToolUseBlock
from claudette.core import *

### Define Model 

In [5]:
model = models[-1]

### Define Tools

In [6]:
bodhicast_spec = "https://api.bodhicast.com/openapi.json"

In [7]:
def get_bodhicast_spec(
) -> dict: # OpenAPI Specification
    "Retrieves the OpenAPI specification for the bodhicast API"
    print(f'- Retrieving bodhicast API spec.')
    res = requests.get(bodhicast_spec)
    return res.json()

In [8]:
get_bodhicast_spec()

- Retrieving bodhicast API spec.


{'openapi': '3.1.0',
 'info': {'title': 'FastAPI', 'version': '0.1.0'},
 'paths': {'/': {'get': {'summary': 'Read Root',
    'operationId': 'read_root__get',
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'schema': {}}}}}}},
  '/forecasts/spots/{date}/{spot_lat}/{spot_lng}': {'get': {'summary': 'Get Forecast By Lat Lng',
    'description': "Retrieve wave forecasts for a specific spot based on date and coordinates.\n\n- This function creates a PostGIS point using the provided latitude (spot_lat) and longitude\n  (spot_lng) as the origin.\n- It then calculates the nearest data point by distance to this origin.\n- Forecasts where significant combined swell and wind wave height (swh) values\n  are null are excluded, as null values indicate land areas rather than water.\n- Additional logic will need to be added to handle rolling forecast updates\n  if fetching from NOAA multiple times per day\n\nArgs:\n    date (str): The date in the f

In [9]:
tools = [get_bodhicast_spec]
chat = Chat(model, tools=tools)

In [10]:
r = chat('What endpoint should i use for adding spots in bodhicast?')
print(r.stop_reason)
r.content

- Retrieving bodhicast API spec.
tool_use


[ToolUseBlock(id='toolu_01Va2i2XbsY9b1BEoxqbJMAK', input={}, name='get_bodhicast_spec', type='tool_use')]

In [11]:
r = chat()

In [12]:
contents(r)

'Based on the OpenAPI specification, the endpoint you should use for adding spots in bodhicast is `/addspot`. This endpoint accepts a `SpotCreate` model as the request body, which includes the latitude, longitude, spot name, and street address of the new spot.\n\nThe relevant details are:\n\n- Endpoint: `/addspot`\n- HTTP Method: `POST`\n- Request Body: `SpotCreate` model with the following properties:\n    - `lat`: latitude of the spot\n    - `lng`: longitude of the spot\n    - `spot_name`: name of the spot\n    - `street_address`: street address of the spot\n- Response: A dictionary with a message indicating the successful creation of the spot.\n\nSo, to add a new spot, you would make a POST request to the `/addspot` endpoint with a JSON payload containing the spot details.'

In [13]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Can you provide me with an example of the structure I should use for the create spot endpoint in bodhicast?', trace_func=print)

- Retrieving bodhicast API spec.
Message(id='msg_018UKYEaupjV8Fva98icXw54', content=[ToolUseBlock(id='toolu_01VjHmR2tsDotsBWfyPusCo2', input={}, name='get_bodhicast_spec', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 374; Out: 41; Total: 415)
- Retrieving bodhicast API spec.
Message(id='msg_01CWvPoS8tdwzSXD8tXGSZuY', content=[TextBlock(text='Based on the OpenAPI specification, the structure for the create spot endpoint in bodhicast should be as follows:', type='text'), ToolUseBlock(id='toolu_013f3P2WLZukbuvSWXT7ZEX6', input={}, name='get_bodhicast_spec', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 2628; Out: 67; Total: 2695)
- Retrieving bodhicast API spec.
Message(id='msg_01CGKQVMqeaZrPChSUCLYFhF', content=[TextBlock(text='The structure for the create spot endpoint in bodhicast should be as follow

In [14]:
print(r.content[0].text)

The create spot endpoint in bodhicast expects a JSON request body with the following structure:

```json
{
    "lat": 36.83055459542353,
    "lng": -75.96764801341773,
    "spot_name": "1st Street Jetty",
    "street_address": "Virginia Beach, Va 23451"
}
```

The request body should be a JSON object with the following properties:

- `lat`: a number representing the latitude of the spot
- `lng`: a number representing the longitude of the spot
- `spot_name`: a string representing the name of the spot
- `street_address`: a string representing the street address of the spot

The endpoint will return a JSON response with a message indicating the successful creation of the spot, like this:

```json
{
    "message": "Spot successfully created"
}
```


### Get a few random spots

In [15]:
bodhicast_url_base = 'https://api.bodhicast.com'

In [16]:
spots_url = f'{bodhicast_url_base}/spots'

In [17]:
spots = requests.get(spots_url)

In [18]:
spot_list = spots.json()

In [19]:
spot_sample = np.random.choice(spot_list, 5, replace=False)

In [20]:
spot_sample

array([{'id': 830, 'latitude': 41.145758, 'longitude': -71.59174, 'spot_name': 'Black Rock Point', 'street_address': 'New Shoreham, Rhode Island'},
       {'id': 361, 'latitude': 39.6458, 'longitude': -74.18338, 'spot_name': 'Borough of Ship Bottom', 'street_address': 'Beach Haven, New Jersey'},
       {'id': 760, 'latitude': 41.315015, 'longitude': -70.76408, 'spot_name': 'Squibnocket', 'street_address': 'Gay Head, Massachusetts'},
       {'id': 945, 'latitude': 33.90129336911256, 'longitude': -118.42184, 'spot_name': 'El Porto', 'street_address': 'Manhattan Beach, California'},
       {'id': 395, 'latitude': 40.113, 'longitude': -74.028, 'spot_name': 'Manasquan Inlet', 'street_address': 'Manasquan, New Jersey'}],
      dtype=object)

### Open Meteo Weather

In [21]:
from string import Template

In [22]:
temperature_template = Template('https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m')

In [23]:
temperature_template.template

'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m'

### Setup Db

In [24]:
import psycopg

In [25]:
conn_info = "dbname='postgres' user='postgres' password='postgres' host='localhost' port='5432'"


In [26]:
with psycopg.connect(conn_info) as conn:
    with conn.cursor() as cur:
        cur.execute("create extension if not exists vector")
        # rows = cur.fetchall()
        # for row in rows:
            # print(row)
        cur.execute("""create table if not exists spots(
                    id serial primary key
                    spot_name                     

                    )""")

SyntaxError: syntax error at or near "name"
LINE 3:                     name
                            ^